---
title: "Telescopes Are Tries: A Dependent Type Shellac on SQLite" 
date: 2025-06-19
---


It seems to me that telescopes, the dependently typed notion of context, is more central to the topic of dependent types than the dependent types are.

A telescope is a sequence of variables and their type such that later types in the sequence can depend on the earlier variables.

`[x : A, y : B(x), z : C(x,y)]`

These are the things often written as `\Gamma$ in a typing judgement $\Gamma \vash t : A$ if you've seen such a thing. Another example might be `[A : Type, n : Nat, x : Vec n A]`` for a having a list that is type constrained to be of length `n` in your current context.


I have continued to think about this blog post https://www.philipzucker.com/frozenset_dtt/ which I rather like. By being vague and borrowing from the metalevel of python, I can kind of outline combinators that describe the simple finite set theoretic model of dependent type theory.

The connection I made in that blog post is that this is very similar to the binding structure you can have with nested for loops

```python
for x in A:
    for y in B(x):
        for z in C(x,y):
            pass
```

Then, somewhat vaguely, the set of python contexts that appear inside that loop represents of models the syntax of the telescope.

On a walk I realized really that there is a more straightforward and less weird direct "thing" we can talk about to represent telescopes: Tries.

Tries can be implemented multiple ways, but one way is as nested dictionaries. 
```python
{ x : {  y : {z : () for z in C(x,y)}  for y in B(x) }  for x in A}
```


In [24]:
from enum import Enum
Bool = frozenset([True, False])
RGB = Enum("RGB", "R G B")
#RGB = frozenset(rgb)
Unit = frozenset([()])

ex1 = {x :{ y : str(y) for y in (RGB if x else Unit)} for x in Bool } 
ex1


{False: {(): '()'},
 True: {<RGB.R: 1>: 'RGB.R', <RGB.G: 2>: 'RGB.G', <RGB.B: 3>: 'RGB.B'}}

In Lean, this example written as a function might look like

In [ ]:
%%file /tmp/tele.lean
inductive RGB where
    | R : RGB
    | G : RGB
    | B : RGB

-- You would have a tough time writing this exact program in Haskell or Ocaml
def ex1 (x : Bool) (y : if x then RGB else Unit) : String :=
    match x with
    | true => match y with
        | RGB.R => "Red"
        | RGB.G => "Green"
        | RGB.B => "Blue"
    | false => match y with
        | () => "Unit"

def main : IO Unit := do
    IO.println (ex1 true RGB.R)
    IO.println (ex1 true RGB.G)
    IO.println (ex1 true RGB.B)
    IO.println (ex1 false ())

Overwriting /tmp/tele.lean


In [29]:
! lean --run /tmp/tele.lean

Red
Green
Blue
Unit


Tries are mappings from their keys to their values. Tries are themselves kind of the set of their keys. Any mapping data structure is kind of a set if you just put `()` as the held value.

In [6]:
def lookup(trie, key):
    subtrie = trie
    for v in key:
        subtrie = subtrie.get(v)
        if trie is None:
            return None
    return subtrie

lookup( ex1 , [True, RGB.R])

<RGB.R: 1>

## Category of Tries

If we lookup a trie with an incomplete key, that's kind of like currying the trie. The value the trie mapping maps to from that perspective is another trie. 

If you makes tries map to trie keys, tries actually can be composed. They form a category. This category corresponds to context mappings in type theory https://www.philipzucker.com/tele/  https://proofassistants.stackexchange.com/questions/2557/what-is-a-context-mapping-in-dependent-type-checking


In [ ]:
#type trie_morph = tuple["trie", int] # a trie morphism is a trie and an integer saying at which point the domain is separate from the codomain. Maybe carrying more dom/cod data would be better.
type trie_morph = object

def id_(trie0, n) -> trie_morph:
    def worker(trie, curkey):
        if trie == ():
            return curkey
        else:
            return {k: worker(v, curkey + [k]) for k, v in trie.items()}

def items(trie):
    def worker(trie, curkey):
        if not isinstance(trie, dict):
            yield (curkey, trie)
        else:
            for k, v in trie.items():
                yield from  worker(v, curkey + [k]) 
    return worker(trie, [])

def trie_map(trie, f):
    def worker(trie):
        if not isinstance(trie, dict):
            return f(trie)
        else:
            return {k: worker(v) for k, v in trie.items()}
    return worker(trie)

def trie_map_with_key(trie, f):
    def worker(trie, curkey):
        if not isinstance(trie, dict):
            return f(curkey, trie)
        else:
            return {k: worker(v, curkey + [k]) for k, v in trie.items()}
    return worker(trie, [])

def id1(trie0): # alternative way of writing id
    trie_map_with_key(trie0, lambda k, x: k)

def compose(trie0 : trie_morph, trie1 : trie_morph) -> trie_morph:
    return trie_map(trie0, lambda x: lookup(trie1, x))

The judgements that go under a context are

- $\Gamma \vdash A type$ - A trie with keys $\Gamma$ that has a type (frozenset) as it's held value
- $\Gamma \vdash t : A$ - A trie with keys $\Gamma$ that has the pair of a type and a value in the type as it's held value.


The things to the right of $\vdash$ should be the values being held in the trie.

This helps me reconcile for example that `True` and `False` aren't the only things in Bool if you're not in an empty context like `x : Bool |- t : Bool`. This is because there are tries that have `True` at every leaf, which is sort of the trie-lift or trie-const form of bool, but there are also tries of course who's value depends on which branch of the trie you're in. These trie correspond to terms `t` that depend on the context variables.


In [ ]:
def const_trie(trie, ntrie, value):
    if ntrie <= 0:
        return value
    else:
        return {k: const_trie(subtrie, ntrie - 1, value) for k,subtrie in trie.items()}
const_trie(ex1, 2, 42)

{False: {(): 42}, True: {<RGB.R: 1>: 42, <RGB.G: 2>: 42, <RGB.B: 3>: 42}}

The thing that tries also triggers in my brain is database queries and worst case optimal join https://arxiv.org/pdf/2301.10841 which also are intimately connected to `for` loops.

There is also strong correspondence between basic SQL queries and `for` loops. https://www.philipzucker.com/sql_graph_csp/

Basically, the for loops correspond to the `FROM` statements, `if` statements correspond to `WHERE` clauses, and `yield` corresponds to `SELECT`

Relations `Set (A,B)` are the same thing (isomorphic to) functions/dictionaries to sets aka multivalued functions.

In [10]:
from collections import defaultdict
rel1 = {(1,True,2), (2,False,3), (2,False,4)}
fun1 = {(1,True) : {2},  (2,False) : {3, 4}}

def rel_to_fun(rel):
    fun = defaultdict(set)
    for xs in rel:
        key, res = xs[:-1], xs[-1]
        fun[key].add(res)
    return fun

rel_to_fun(rel1)


defaultdict(set, {(2, False): {3, 4}, (1, True): {2}})

In [11]:
def fun_to_rel(fun):
    rel = set()
    for key, res in fun.items():
        for r in res:
            rel.add(key + (r,))
    return rel
fun_to_rel(fun1)

{(1, True, 2), (2, False, 3), (2, False, 4)}

So we can take the convention on SQL tables that the table `C` with 3 columns is actually representing a multivalued function from the first 2 columns to the third.


Any telescope can be rewritten as a conjunctive query.

`[x : A, y : B(x), z : C(x,y)]` becomes $A(x) \land B(x,y) \land C(x,y,z)$.
This can in turn can be written as the SQL query. SQL binds rows rather than the elements in the rows like a conjunctive query / datalog body does. It is however quite mechanical to translate them.  https://www.philipzucker.com/tiny-sqlite-datalog/

```SQL
FROM A as a 
FROM B as b 
FROM C as c 
WHERE
a.v0 = b.v0 AND
b.v1 = c.v1 AND
a.v0 = c.v0
```


Vice versa can also be achieved in various ways.

$T(x,y) \land T(y,z) \land T(z,x)$ becomes `[x : Int, y : Int, z : Int, p1 : T(x,y), p2 : T(y,z), p3 : T(z,x)]`  or we could try to move the telescope around a little, which corresponds to lifting up loop invariant code in a for loop to `[x : Int, y : Int, p1 : T(x,y), z : Int, p2 : T(y,z), p3 : T(z,x)]` 

It seems in general it is eaiser to convert a telescope to a nice conjunctive query form than vice versa. Conjuctive queries have no restrictions on there being a nice ordering to the variables. There may not be a nice ordering or finding it is a task.

Telescopes and Conjucttive queries kind of correspond to two "standard forms" you might like your logical formulas to appear as

- "Sequent form" $\forall x y z ... , ( A \land B \land C ...)  \implies P$
- Telescope form $\forall x, A(x) \implies (\forall y, B(x,y) \implies (... \implies P))$ 

Always working in these two forms let's you form Hilbert style combinators that corresponds to sequence calculus rules or rules that look more like dependent type theory rules.


Subsingleton/Propositions https://ncatlab.org/nlab/show/subsingleton are tables with an actual functional dependency between the inputs and output. There is either no key or just one.

This is kind of a cute way to replace an `if` statements with a `for` loop. 

Analogously, in SQL, a `WHERE` clause can be replaced by a table that is either empty or has one row for that key.





In [20]:
if True:
    print("its true")

if False:
    print("shouldn't print")

its true


In [21]:
for x in ["just one thing"]:
    print("also true")

for x in []:
    print("shouldn't print")

also true


An interesting topic in datalog / database is provenance, knowing how a fact ended up in the database. This is a richer notion of truth value / proof object that just being in there or not.

https://arxiv.org/abs/2202.10766
https://souffle-lang.github.io/provenance


In [ ]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("CREATE TABLE rel1 (x,y,z)")
cur.executemany("INSERT INTO rel1 VALUES (?,?,?)", rel1)
cur.execute("SELECT * FROM rel1").fetchall()

In [32]:
from sympy import *
x, y, z = symbols('x y z')
# make a semiring mod x**2 - 1
from sympy.abc import x
from sympy import QQ
QQ.old_poly_ring(x).quotient_ring(QQ.old_poly_ring(x).ideal(x**2))
QQ.old_poly_ring(x).quotient_ring([x**2])

s, c = symbols('s, c')
QQ.old_poly_ring(s, c) / [s**2 + c**2 - 1]

QQ[s,c]/<c**2 + s**2 - 1>

https://github.com/true-grue/python-dsls/blob/main/datalog/datalog_test.py

In [114]:
#def Tele(x, A, cb):
import ast
#print(ast.dump(ast.parse("[x in int, b in int] >= t in A", mode="eval"), indent=4))
ast.dump(ast.parse("(x in A, y in B) -> t == t1 in A", mode="func_type"), indent=4)
print(ast.dump(ast.parse("x : A; y : B; t == t1 in A", mode="exec"), indent=4)) # yield, asset
#print(ast.dump(ast.parse("[x in int, b in int] => x in B")))


Module(
    body=[
        AnnAssign(
            target=Name(id='x', ctx=Store()),
            annotation=Name(id='A', ctx=Load()),
            simple=1),
        AnnAssign(
            target=Name(id='y', ctx=Store()),
            annotation=Name(id='B', ctx=Load()),
            simple=1),
        Expr(
            value=Compare(
                left=Name(id='t', ctx=Load()),
                ops=[
                    Eq(),
                    In()],
                comparators=[
                    Name(id='t1', ctx=Load()),
                    Name(id='A', ctx=Load())]))],
    type_ignores=[])


What does cross stage get us? We get to produce python code.


In [115]:
import z3
# cross stage persistence
def cross(e : z3.ExprRef) -> str:
    return f"z3.deserialize(\"{e.serialize()}\")".replace("\n", "")

def mypow(n : int, e : z3.ExprRef) -> str:
    if n == 0:
        return cross(e)
    else:
        return f"{cross(e)} * {mypow(n - 1, e)}"

eval(cross(z3.Int('x')))
eval(mypow(3, z3.Int('x')))

def mypow1(n, e):
    if n == 0:
        return e
    else:
        return e * mypow1(n - 1, e)
mypow1(3, z3.Int('x'))

x*x*x*x

In [96]:
z3.deserialize("(declare-fun F (Int) Bool)\n(declare-fun x () Int)\n(assert (F x))\n")

x

In [ ]:
print(ast.dump(ast.parse("""
for x in range(7): # type: int
    print(x)
""",type_comments=True), indent=4))
ast.unparse
ast.literal_eval("(1,2)")
ast.literal_eval("")

Module(
    body=[
        For(
            target=Name(id='x', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
                args=[
                    Constant(value=7)],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='x', ctx=Load())],
                        keywords=[]))],
            orelse=[],
            type_comment='int')],
    type_ignores=[])


(1, 2)